In [1]:
import os
import pickle
import pandas as pd
import numpy as np
from datetime import datetime
import statistics
from math import sqrt
from numpy import log as ln
from tqdm import tqdm
from pymongo import MongoClient
import statsmodels.api as sm
from sklearn.utils import all_estimators
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv('dataset.csv')

In [3]:
df

,_id,Name,Price,Number of Reviews,Category,Score,Sub-Category,SubSub-Category,Reviews,Link,Popularity,Date
0,63c2a89a0e579974320b7703,Women's Body Warm Thermal Suit For Winter,Rs. 550,4 Ratings,Fashion,5.0/5,Men,Clothing,[[]],https://www.daraz.pk/products/womens-body-warm...,48,2023-01-14
1,63c2a89b0e579974320b7704,Baby Waterproof Diaper Travel Changing Urine T...,Rs. 160,34 Ratings,Mother & Baby,4.4/5,Diapering & Potty,"Changing Tables, Pads & Kits",[[]],https://www.daraz.pk/products/-i350438488-s178...,55,2023-01-14
2,63c2a89b0e579974320b7705,High Quality Thermal Fleece Winter Inner Suit...,"Rs. 1,090",70 Ratings,Fashion,4.4/5,Men,Clothing,[[]],https://www.daraz.pk/products/-i255081169-s188...,48,2023-01-14
3,63c2a89b0e579974320b7706,Pack of 2 Thermal Winter FLEECE NECK Warmer,Rs. 95,154 Ratings,Fashion,4.4/5,Men,Clothing,[[]],https://www.daraz.pk/products/2-i189602500-s13...,48,2023-01-14
4,63c2a89b0e579974320b7707,Winter Complete Thermal Suit for Men (Inner We...,Rs. 620,5 Ratings,Fashion,5.0/5,Men,Clothing,[[]],https://www.daraz.pk/products/-i376169651-s186...,48,2023-01-14
...,...,...,...,...,...,...,...,...,...,...,...,...
2089522,6435a70790a036ab7fb1ea25,MiniMe 6 Pcs Infant Baby Diaper Changing Mat B...,Rs. 645,7 Ratings,Mother & Baby,4.9/5,Diapering & Potty,"Changing Tables, Pads & Kits",[[]],https://www.daraz.pk/products/6-i357577741-s17...,50,2023-04-11
2089523,6435a70890a036ab7fb1ea26,Foldable Baby Diaper Caddy Organiser Removable...,"Rs. 2,892",1 Ratings,Mother & Baby,5.0/5,Diapering & Potty,"Changing Tables, Pads & Kits",[[]],https://www.daraz.pk/products/-i178002218-s135...,50,2023-04-11
2089524,6435a70890a036ab7fb1ea27,50*70 CM Baby Infant Diaper Cute Cartoon Anima...,"Rs. 1,110",3 Ratings,Mother & Baby,5.0/5,Diapering & Potty,"Changing Tables, Pads & Kits",[[]],https://www.daraz.pk/products/5070-i335256930-...,50,2023-04-11
2089525,6435a70f90a036ab7fb1ea28,HelloKimi Baby Changing Pads Foldable Washable...,"Rs. 1,870",24 Ratings,Mother & Baby,4.8/5,Diapering & Potty,"Changing Tables, Pads & Kits",[[]],https://www.daraz.pk/products/-i180874124-s135...,50,2023-04-11


In [4]:
len(df)

2089527

In [4]:
df["Number of Reviews"] = df["Number of Reviews"].str.replace("Ratings","")
df["Number of Reviews"] = df["Number of Reviews"].str.replace("No","0")
df['Number of Reviews'] = df['Number of Reviews'].astype('int')
df["Score"] = df["Score"].str.replace("/5","")
df['Score'] = df['Score'].astype('float')

In [5]:
data = pd.DataFrame()
data["Categoric Mapping"] = "Categoric Mapping"
data["Categoric Mapping"] = df["Date"] + "_" + df["Category"] + "/" + df[ "Sub-Category"] + "/" + df["SubSub-Category"]
data["Mean Reviews"] = df["Number of Reviews"]
data["Mean Score"] = df["Score"]
data["Popularity"] = df["Popularity"]

In [6]:
value_counts = dict(data["Categoric Mapping"].value_counts())

In [7]:
data = data.groupby("Categoric Mapping").agg({"Mean Reviews":"sum", 
                                              "Mean Score":"sum", 
                                              "Popularity":"max"})
data.reset_index(inplace = True)

In [8]:
keys = list(value_counts.keys())
for i in tqdm(range(len(data))):
    temp = data.loc[i, "Categoric Mapping"]
    try:
        data.loc[i, "Mean Reviews"] = data.loc[i, "Mean Reviews"] / value_counts[temp]
        data.loc[i, "Mean Score"] = data.loc[i, "Mean Score"] / value_counts[temp]
    except:
        pass

100%|██████████| 52955/52955 [00:13<00:00, 3853.28it/s]


In [9]:
for i in tqdm(range(len(data))):
    if " 00:00:00" in data.loc[i, "Categoric Mapping"]:
        data.loc[i, "Categoric Mapping"] = data.loc[i, "Categoric Mapping"].replace(" 00:00:00", "")

100%|██████████| 52955/52955 [00:00<00:00, 101914.16it/s]


In [10]:
dates = []
for i in range(len(data)):
    date = data["Categoric Mapping"][i].split("_")[0]
    dates.append(datetime.strptime(date, "%Y-%m-%d"))
data["Date"] = dates
dates = np.unique(dates)

In [11]:
data['Rate of Change'] = None
data['Mean Reviews'] = data['Mean Reviews'] + 1
df = data

# Label = {[(1 + p) × (1 + ΔR)] x μ(S)} / σ(N)

In [12]:
def label_data_set(df):
    df['Rate of Change'] = None
    df['Mean Reviews'] = df['Mean Reviews'] + 1

    for i in range(len(df)):
        df.loc[i, "Categoric Mapping"] = df.loc[i, "Categoric Mapping"].split("_")[-1]

    for i in tqdm(range(len(df))):
        cat_name = df.loc[i, 'Categoric Mapping']
        lis = df.loc[df['Categoric Mapping'] == cat_name].index.tolist()
        index = lis.index(i)
        if (index == 0):
            df.loc[i, 'Rate of Change'] = 0
        else:
            index_last = index - 1
            reviews = [int(df.loc[lis[index - 1], 'Mean Reviews']), int(df.loc[lis[index], 'Mean Reviews'])]
            reviews_series = pd.Series(reviews)
            df.loc[i, 'Rate of Change'] = list(reviews_series.pct_change())[1]

    df['Half Label'] = None
    for i in tqdm(range(len(df))):
        value = 1 + float(df.loc[i, 'Popularity'])
        value2 = 1 + float(df.loc[i, 'Rate of Change'])
        value = value * value2
        df.loc[i, 'Half Label'] = value
        
    df['Standard Dev'] = None
    meann = {}
    for i in tqdm(list(df['Date'].unique())):
        m = df.loc[df['Date'] == i, 'Mean Reviews'].mean()
        meann[i] = m
    for i in tqdm(list(df['Date'].unique())):
        val = statistics.stdev(list(df.loc[df['Date'] == i, 'Mean Reviews']))
        df.loc[df['Date'] == i, 'Standard Dev'] = val
        
    df['Label'] = None
    for i in tqdm(range(len(df))):
        val1 = df.loc[i, 'Half Label'] * df.loc[i, 'Mean Score']
        val1 = val1 / df.loc[i, 'Standard Dev']
        df.loc[i, 'Label'] = val1
  
    df.drop(['Mean Reviews', 'Half Label'], axis = 1, inplace = True)
    return df

In [13]:
data = label_data_set(df)

 14%|█▍        | 7610/52955 [00:36<03:40, 205.72it/s]


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame()
df["Category"] = data["Categoric Mapping"].str.split("/", expand=True)[0]
df["Sub-Category"] = data["Categoric Mapping"].str.split("/", expand=True)[1]
df["Sub-Sub-Category"] = data["Categoric Mapping"].str.split("/", expand=True)[2]
df["Year"] = data["Date"].dt.year
df["Month"] = data["Date"].dt.month
df["Day"] = data["Date"].dt.day
df["Week_Day"] = data["Date"].dt.strftime('%A')
df['Rate of Change'] = data['Rate of Change']
df['Standard Dev'] = data['Standard Dev']
df['Popularity'] = data['Popularity']
df["Label"] = data["Label"]
df['Popularity'] = data['Popularity']
df.sort_index(axis = 0)

,Category,Sub-Category,Sub-Sub-Category,Year,Month,Day,Week_Day,Rate of Change,Standard Dev,Popularity,Label
0,Bags and Travel,Men Bags,Backpacks,2023,1,14,Saturday,0,174.916509,26,0.720472
1,Bags and Travel,Men Bags,Business Bags,2023,1,14,Saturday,0,174.916509,19,0.469938
2,Bags and Travel,Men Bags,Crossbody Bags,2023,1,14,Saturday,0,174.916509,33,0.896085
3,Bags and Travel,Men Bags,Messenger Bags,2023,1,14,Saturday,0,174.916509,0,0.026298
4,Bags and Travel,Men Bags,Wallets & Accessories,2023,1,14,Saturday,0,174.916509,20,0.559767
...,...,...,...,...,...,...,...,...,...,...,...
52950,Watches Sunglasses Jewellery,Jewellery,Men,2023,4,11,Tuesday,0.014599,277.52375,39,0.636126
52951,Watches Sunglasses Jewellery,Jewellery,Women,2023,4,11,Tuesday,0.0,277.52375,32,0.444496
52952,Watches Sunglasses Jewellery,Watches,Kids,2023,4,11,Tuesday,-0.122302,277.52375,13,0.19202
52953,Watches Sunglasses Jewellery,Watches,Men,2023,4,11,Tuesday,-0.018957,277.52375,8,0.129965


In [ ]:
def reassign_dates(df):
    df = df.copy()
    unique_dates_df = df[['Year', 'Month', 'Day']].drop_duplicates()
    unique_dates = unique_dates_df.values.tolist()
    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    df['Date'] = df['Date'] - pd.DateOffset(days=len(unique_dates)+1)
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['Week_Day'] = df['Date'].dt.strftime('%A')
    df = df.drop('Date', axis=1)
    new_df = df.copy()
    return new_df

def generate_data(df, span):
    total_data = []
    df_copy = df.copy()
    for instance in range(span):
        df_copy = reassign_dates(df_copy)
        total_data.append(df_copy)
    total_data.reverse()
    combined_df = pd.concat(total_data)
    combined_df = combined_df.reset_index(drop=True)
    return combined_df

In [ ]:
def remove_outliers_using_zscore(df, column_name, z_threshold=1.96):
    mean = df[column_name].mean()
    std = df[column_name].std()

    z_scores = (df[column_name] - mean) / std
    filtered_df = df[np.abs(z_scores) <= z_threshold]
    return filtered_df

In [ ]:
# PREPROCESSING
df['Category_Sub-Category_Sub-Sub-Category'] = df['Category'] + '_' + df['Sub-Category'] + '_' + df['Sub-Sub-Category']
threshold = 10 * (df['Category_Sub-Category_Sub-Sub-Category'].value_counts().iloc[0] / 100)
df = df.groupby('Category_Sub-Category_Sub-Sub-Category').filter(lambda x: len(x) >= threshold)
df = df.drop('Category_Sub-Category_Sub-Sub-Category', axis=1)
df = remove_outliers_using_zscore(df, "Label", z_threshold = 1.96)
df.reset_index(drop=True, inplace=True)
#SETTING ALL POPULARITY VALUES TO 0 (TEMPORARILY)
df['Popularity'] = 0

In [ ]:
df

,Category,Sub-Category,Sub-Sub-Category,Year,Month,Day,Week_Day,Rate of Change,Standard Dev,Popularity,Label
0,Bags and Travel,Men Bags,Backpacks,2023,1,14,Saturday,0,174.916509,0,0.720472
1,Bags and Travel,Men Bags,Business Bags,2023,1,14,Saturday,0,174.916509,0,0.469938
2,Bags and Travel,Men Bags,Crossbody Bags,2023,1,14,Saturday,0,174.916509,0,0.896085
3,Bags and Travel,Men Bags,Messenger Bags,2023,1,14,Saturday,0,174.916509,0,0.026298
4,Bags and Travel,Men Bags,Wallets & Accessories,2023,1,14,Saturday,0,174.916509,0,0.559767
...,...,...,...,...,...,...,...,...,...,...,...
52483,Watches Sunglasses Jewellery,Jewellery,Men,2023,4,11,Tuesday,0.014599,277.52375,0,0.636126
52484,Watches Sunglasses Jewellery,Jewellery,Women,2023,4,11,Tuesday,0.0,277.52375,0,0.444496
52485,Watches Sunglasses Jewellery,Watches,Kids,2023,4,11,Tuesday,-0.122302,277.52375,0,0.19202
52486,Watches Sunglasses Jewellery,Watches,Men,2023,4,11,Tuesday,-0.018957,277.52375,0,0.129965


In [ ]:
csv_output_directory = "../Back-End/"
df.to_csv(os.path.join(csv_output_directory, 'processed_data.csv'), index=False)

In [ ]:
df = pd.read_csv("../Back-End/processed_data.csv")

In [ ]:
# #ADDING MORE DATA
# generated_data = generate_data(df, 5)
# df = pd.concat([generated_data, df])
# df = df.reset_index(drop=True)

In [ ]:
generated_data = pd.read_csv("final_generated_dataset.csv")

In [ ]:
result = pd.concat([generated_data, df], axis=0)
df = result

In [ ]:
df

,Category,Sub-Category,Sub-Sub-Category,Year,Month,Day,Week_Day,Rate of Change,Standard Dev,Popularity,Label
0,Bags and Travel,Men Bags,Backpacks,2020,8,27,Thursday,1.935701,174.916509,22,1.527749
1,Bags and Travel,Men Bags,Business Bags,2020,8,27,Thursday,0.321704,174.916509,14,0.448578
2,Bags and Travel,Men Bags,Crossbody Bags,2020,8,27,Thursday,3.200402,174.916509,21,2.090864
3,Bags and Travel,Men Bags,Messenger Bags,2020,8,27,Thursday,0.162291,174.916509,0,0.026298
4,Bags and Travel,Men Bags,Wallets & Accessories,2020,8,27,Thursday,0.141834,174.916509,19,0.516708
...,...,...,...,...,...,...,...,...,...,...,...
52483,Watches Sunglasses Jewellery,Jewellery,Men,2023,4,11,Tuesday,0.014599,277.523750,0,0.636126
52484,Watches Sunglasses Jewellery,Jewellery,Women,2023,4,11,Tuesday,0.000000,277.523750,0,0.444496
52485,Watches Sunglasses Jewellery,Watches,Kids,2023,4,11,Tuesday,-0.122302,277.523750,0,0.192020
52486,Watches Sunglasses Jewellery,Watches,Men,2023,4,11,Tuesday,-0.018957,277.523750,0,0.129965


In [ ]:
weekday_encoding = {
    "Monday" : 1,
    "Tuesday" : 2,
    "Wednesday" : 3,
    "Thursday" : 4,
    "Friday" : 5,
    "Saturday" : 6,
    "Sunday" : 7
}

In [ ]:
df['Rate of Change'] = df['Rate of Change'].astype(float)
df['Standard Dev'] = df['Standard Dev'].astype(float)
df['Label'] = df['Label'].astype(float)

In [ ]:
le_category = LabelEncoder()
le_sub_category = LabelEncoder()
le_sub_sub_category = LabelEncoder()

df['Category'] = le_category.fit_transform(df['Category'])
df['Sub-Category'] = le_sub_category.fit_transform(df['Sub-Category'])
df['Sub-Sub-Category'] = le_sub_sub_category.fit_transform(df['Sub-Sub-Category'])
df['Week_Day'] = df['Week_Day'].map(weekday_encoding)
    
#SETTING ALL POPULARITY VALUES TO 0 (TEMPORARILY)
# df['Popularity'] = 0

X = df.copy()
y = X.pop('Label')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_pred - y_true) / ((np.abs(y_pred) + np.abs(y_true)) / 2))) * 100

def mean_bias_deviation(y_true, y_pred):
    return np.mean((y_pred - y_true) / y_true) * 100

def mean_directional_accuracy(y_true, y_pred):
    return np.mean(np.sign(np.diff(y_true)) == np.sign(np.diff(y_pred))) * 100

dt_r = RandomForestRegressor(random_state=0)
dt_r.fit(X_train, y_train)
y_pred = dt_r.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
smape = symmetric_mean_absolute_percentage_error(y_test, y_pred)
mbd = mean_bias_deviation(y_test, y_pred)
mda = mean_directional_accuracy(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
print("mean sqaured error is", mse)
print("root mean sqaured error is", rmse)
print("coefficent of determination is", r2)

mean sqaured error is 0.1129589057385232
root mean sqaured error is 0.3360935966937234
coefficent of determination is 0.9899599263796699


In [ ]:
output_directory = "C:\\Users\\Hassa\\OneDrive\\Desktop\\PRODUCT_SCOUT\\Back-End"

with open(os.path.join(output_directory, 'model.pkl'), 'wb') as f:
    pickle.dump(dt_r, f)

with open(os.path.join(output_directory, 'cat_encoder.pkl'), 'wb') as f:
    pickle.dump(le_category, f)

with open(os.path.join(output_directory, 'sub_cat_encoder.pkl'), 'wb') as f:
    pickle.dump(le_sub_category, f)

with open(os.path.join(output_directory, 'subsub_cat_encoder.pkl'), 'wb') as f:
    pickle.dump(le_sub_sub_category, f)